# We start off by reading the data

In [ ]:
import pandas as pd
import plotly.graph_objects as go

import shared

df = pd.read_pickle('data/df_vac_BY.pkl')
pd.options.plotting.backend = "plotly"
# colors
import plotly.express as px
from itertools import cycle
palette = shared.get_palette()



## Let's take a look

In [ ]:
df.tail(5)

In [ ]:
current_official_doses = df.tail(1)['dosen_kumulativ']
#current_official_doses.values[0]
current_official_doses


In [ ]:
df.set_index('date')['shots_sum'].plot.line(width=1000, height=500, title='Impfungen Bayern').show()

### Average doses of the last days

In [ ]:
def avg_doses_of_last_x_days(last_days):
    data = []
    for i in range(1, last_days):
        data.append([i, int(df.tail(i)['shots_today'].sum() /i)])

    a = pd.DataFrame(columns=["Last x days", "Average vacs"], data=data)

    fig = go.Figure(data=[
        go.Bar(name='Three',x=a['Last x days'], y=a['Average vacs'])

    ])
    fig.update_layout(
        width=1000,
        height=350,
        title='Average doses of the last x days'
    )
    fig.show()
    return a


In [ ]:
a = avg_doses_of_last_x_days(100)
a.head(7)

In [ ]:
a = avg_doses_of_last_x_days(365)



In [ ]:
to_plot = df.set_index('date')

fig = go.Figure()

def add_rolling(fig1, df1, days):
    fig1.add_trace(go.Scatter(x = to_plot.index,
                         y=df1.shots_today.rolling(days).mean(),
                         mode='lines',
                         name=f'Smoothed (rolling {days} day window)',
                         marker_color=next(palette),
                         line=dict( width=4)))

fig.add_trace(go.Scatter(x = to_plot.index,
                         y=to_plot.shots_today,
                         mode='markers',
                         name='Real',
                         line=dict(color='grey', width=1)))



add_rolling(fig, to_plot, 7)
add_rolling(fig, to_plot, 14)
add_rolling(fig, to_plot, 21)
add_rolling(fig, to_plot, 28)
fig.update_layout(
    width=1000,
    height=500,
    yaxis_range=[0,100_000],
    title='Daily Vaccination speed change'
)
fig.show()

## Vaccinations per weekday

In [ ]:

to_plot_all = df.groupby(["weekday_name"])['shots_today'].mean().sort_values()
to_plot_10 = df.tail(7*10).groupby(["weekday_name"])['shots_today'].mean().sort_values()
to_plot_5 = df.tail(7*5).groupby(["weekday_name"])['shots_today'].mean().sort_values()
to_plot_2 = df.tail(7*2).groupby(["weekday_name"])['shots_today'].mean().sort_values()


fig = go.Figure(data=[
    go.Bar(name='complete timeline', y=to_plot_all, x=to_plot_all.index),
    go.Bar(name='10 weeks', y=to_plot_10, x=to_plot_all.index),
    go.Bar(name='5 weeks', y=to_plot_5, x=to_plot_all.index),
    go.Bar(name='2 weeks', y=to_plot_2, x=to_plot_all.index),
])
fig.update_layout(
    width=1000,
    height=500,
    title='Average vaccinations per weekday'
)
fig.show()

## Vaccinations per calendar week

In [ ]:
# tail(length - 5) removes 53rd calendar week
to_plot_sum = df.tail(len(df.index) - 9).groupby(["calendar_week"])['shots_today'].sum()
to_plot_mean = df.tail(len(df.index) - 9).groupby(["calendar_week"])['shots_today'].mean()

fig = go.Figure(data=[
    go.Bar(name='Sum' ,y=to_plot_sum),
    go.Bar(name='Mean per day', y=to_plot_mean),
])
fig.update_layout(
    width=1000,
    height=500,
    title='Vaccinations per calendar week'
)
fig.show()



## Guessing how many vac we'll have next

We have to differentiate between weekend and week days

In [ ]:
def is_next_day_weekend(df):
    next_day = df.tail(1).date + pd.DateOffset(1)
    if next_day.dt.dayofweek.values[0] > 4:
        return True
    else:
        return False


DAYS_TO_LOOK_BACK = 2

import math

def guess_next_days_vacs(df, is_weekend):
    df_filtered = df[df['is_weekend']==is_weekend]
    mean = df_filtered.tail(DAYS_TO_LOOK_BACK)['shots_today'].values.mean()
    return math.ceil(mean)


print(f"Mean of weekdays: {guess_next_days_vacs(df, False)}")
print(f"Mean of weekends: {guess_next_days_vacs(df, True)}")


